In [1]:
from skyfield.api import load, N, W, wgs84, utc, Angle
from datetime import datetime
from pytz import timezone
est = timezone("US/Eastern")
ts = load.timescale()
de421 = load("de421.bsp")
earth, sun, moon = de421["earth"], de421["sun"], de421["moon"]
# 40°37′55″ N  82°58′37″ W
caledonia_wsg = wgs84.latlon(40.63194 * N, 82.9769 * W)
caledonia = earth + caledonia_wsg

In [2]:
def print_observe(from_obj, obj, tt):
    pos = from_obj.at(tt).observe(obj)
    alt, az, d = pos.apparent().altaz()
    ra, dec, d = pos.radec()
    print(f"{obj.target_name.split()[-1]}\nalt: {alt.degrees:.2f}\t az: {az.degrees:.1f}\n ra: {ra.hours:.3f}h\tdec: {dec.degrees:.1f}")

In [3]:
t = ts.now()
#t = ts.from_datetime(datetime.strptime("2025-08-23 06:06 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
lst = caledonia_wsg.lst_hours_at(t)
print(f"{t.tt_strftime()}\nLST: {Angle(hours=lst)} ({lst:.2f}h)")
print_observe(caledonia, sun, t)
print_observe(caledonia, moon, t)

2025-10-08 16:25:53 TT
LST: 12h 03m 03.13s (12.05h)
SUN
alt: 41.56	 az: 161.7
 ra: 12.939h	dec: -6.0
MOON
alt: -25.32	 az: 325.7
 ra: 2.170h	dec: 16.6


In [5]:
from skyfield import almanac
a = ts.from_datetime(datetime.strptime("2025-10-06 00:00 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
b = ts.from_datetime(datetime.strptime("2025-10-15 00:00 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
t, y = almanac.find_discrete(a, b, almanac.moon_phases(de421))

for tm, ty, ph in zip(t.astimezone(est),y,[almanac.MOON_PHASES[yi] for yi in y]):
    print(f"{tm.isoformat(' ', 'seconds')} {ph}")
# almanac.moon_phases([a,b])
rt, y = almanac.find_risings(caledonia, sun, a, b)
st, y = almanac.find_settings(caledonia, sun, a, b)
for rtm, stm in zip(rt,st):
    print(f"{rtm.astimezone(est).date()} {rtm.astimezone(est).time().isoformat('seconds')} / {stm.astimezone(est).time().isoformat('seconds')}")

2025-10-06 23:47:36-04:00 Full Moon
2025-10-13 14:12:41-04:00 Last Quarter
2025-10-06 07:33:52 / 19:05:16
2025-10-07 07:34:54 / 19:03:39
2025-10-08 07:35:57 / 19:02:03
2025-10-09 07:36:59 / 19:00:28
2025-10-10 07:38:02 / 18:58:53
2025-10-11 07:39:06 / 18:57:19
2025-10-12 07:40:09 / 18:55:46
2025-10-13 07:41:13 / 18:54:13
2025-10-14 07:42:18 / 18:52:41
